# One target island (openCV 4.0.1)

## Import modules

Make sure that Python 3 and the following modules (recommended version ID) are installed on your computer before running this cell:

numpy (1.18.1),
sounddevice (0.3.14),
openCV (4.0.1),
tkinter (8.6.8),
xlxswriter (1.2.7),
scipy (1.3.2),
pyfirmata (1.1.0)

In [1]:
import numpy as np                        # Import numpy module
                                          # conda install -c conda-forge python-sounddevice
import sounddevice as sd                  # Import sounddevice module for "real-time" sound playback
import cv2                                # Import opencv module for image processing
print(cv2.__version__)                    # Make sure you have the correct version of openCV 
import tkinter as tk                      # Import Tkinter module for GUIs
import xlsxwriter                         # Import xlsxwriter module for writing a protocol on-the-fly

from scipy.io  import wavfile             # WAV-file import filter
from pyfirmata import Arduino             # Arduino support
from math      import e                   # Euler's number
from collections import OrderedDict

import math                               # Import math module
import time                               # Import time module for time measurements and pausing
import random                             # Import random module for random number generation
import json
import datetime
import os

4.2.0


## Load default session settings

For every particular experimental cofiguration one can copy the 'defaults.json' file as a specific experimental preset and load it here instead of 'defaults.json'.

In [2]:
with open('defaults.json') as json_file:
    cfg = OrderedDict(json.load(json_file))
cfg['experiment_date'] = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print(json.dumps(cfg, indent=4))

{
    "trial_number": 50,
    "session_duration": 3600,
    "trial_duration": 60,
    "start_radius": 67,
    "start_x": 195,
    "start_y": 195,
    "target_radius": 80,
    "target_duration": 5,
    "subject": "003901",
    "experiment_type": "aSIT",
    "background_color": "T",
    "init_duration": 0.2,
    "arena_x": 400,
    "arena_y": 300,
    "arena_radius": 300,
    "experiment_date": "2020-06-11_21-29-29"
}


## Open a starting screen

The next cell will open a window that can be used to configure the experiment by entering the desired number of trials per session (usually limited by the number of rewards the feeder provides without refil), the session and trial durations, the size (here: 134 px in diameter = 17.87 cm) and the XY-position of the starting platform, the size of the target area, and the duration the animal has to spend in the target to recieve a reward. In addition, you can give your experiment a purposeful ID (e.g. subject ID, experiment type and date), provide information about the contrast between arena and subject, and define the duration the animal has to spend in the starting area to initialize a new trial. 

If you choose your target size, always make sure that it is small enough to fit in the arena without overlap with the starting/initialization area. 

The experiment ID you enter in the popup window will automatically be added to the file names of the protocols that will be generated for each session.

**To save your configuration, hit the apply button**. To close the popup window and proceed, hit the continue button.

In [3]:
def show_entry_fields():
    print("EXPERIMENT ID: %s" % experiment_id)
    for key, field in entry_fields.items():
        print("%s: %s" % (labels[key].cget('text'), entry_fields[key].get()))
        
def update_config():
    for key, entry_field in entry_fields.items():
        tp = type(cfg[key])
        cfg[key] = tp(entry_field.get())
    experiment_id = "%s_%s_%s" % (cfg['subject'], cfg['experiment_type'], cfg['experiment_date'])
        
def show_and_update_config():
    update_config()
    show_entry_fields()
    
# Starting screen
master = tk.Tk()
master.title('Experimental parameters')

# Labels
tk.Label(master, text="Instructions: \n 1. Enter parameters (only integers are allowed as numbers) \n 2. Press 'Apply' \n 3. Press 'Continue'").grid(row=0, padx=10, pady=10)
labels = OrderedDict()
labels['trial_number'] = tk.Label(master, text="Trials per session")
labels['session_duration'] = tk.Label(master, text="Session duration [s]")
labels['trial_duration'] = tk.Label(master, text="Trial duration [s]")
labels['start_radius'] = tk.Label(master, text="Radius of the starting platform [pixels]")
labels['start_x'] = tk.Label(master, text="X-position of the starting platform [pixels]")
labels['start_y'] = tk.Label(master, text="Y-position of the starting platform [pixels]")
labels['target_radius'] = tk.Label(master, text="Radius of the target platform [pixels]")
labels['target_duration'] = tk.Label(master, text="Target duration [s]")
labels['subject'] = tk.Label(master, text="Subject")
labels['experiment_type'] = tk.Label(master, text="Experiment type")
labels['background_color'] = tk.Label(master, text="Subject is darker than background [T = True; F = False]")
labels['init_duration'] = tk.Label(master, text="Initialisation Duration [s]")
labels['arena_x'] = tk.Label(master, text="Arena X coordinate [pixels]")
labels['arena_y'] = tk.Label(master, text="Arena Y coordinate [pixels]")
labels['arena_radius'] = tk.Label(master, text="Arena radius [pixels]")
labels['experiment_date'] = tk.Label(master, text="Experiment date")

for i, (key, label) in enumerate(labels.items()):
    label.grid(row=i + 4, padx=5, pady=5)

# Entry fields
entry_fields = OrderedDict()
for i, (key, value) in enumerate(cfg.items()):
    entry_fields[key] = tk.Entry(master)
    entry_fields[key].insert('end', value)
    entry_fields[key].grid(row=i + 4, column=1)

experiment_id = "%s_%s_%s" % (cfg['subject'], cfg['experiment_type'], cfg['experiment_date'])

tk.Button(master, text='Apply', command=show_and_update_config).grid(row=len(entry_fields) + 4, column=0, sticky='s', pady=4)
tk.Button(master, text='Continue', command=master.destroy).grid(row=len(entry_fields) + 4, column=1, sticky='w', pady=4)

tk.mainloop()

## Protocol 1

Run the upcoming cell, to create a session folder (here, inside 'sessions') and to save the chosen experimetal parameters to a JSON-file ("experiment_id_parameters.json").

In [4]:
# This session will be written to this folder
save_to = os.path.join('sessions', experiment_id)
             
if not os.path.exists(save_to):
    os.makedirs(save_to)

# Saves all parameters to a JSON file with the user-defined "Experiment ID" as filename
with open(os.path.join(save_to, experiment_id + '_parameters.json'), 'w') as f:
    json.dump(cfg, f, indent=4)

## Initialize the microcontroller

The next cell Initializes a microcontroller for subsequent hardware control. This is, where you will probably have to get creative yourself, depending on what you would like to do. Here, we use an Arduino Nano. With the channel definitions below, we can later provide differently colored illumination during the experiment (for example to stimulate with colors rather than sound) and trigger two different feeders. 

For the example setup, two automatic fish feeders with 27 feeding slots each were "hacked", so that they can be controlled *via* two additional Arduinos with motor shields. These additional Arduinos drive the feeder motors each time they get a trigger signal from the main Arduino. The two feeders allow the provision of 54 rewards per session. The two feeders were installed at different positions above the arena and are activated alternately, to lower the predictability of where in the arena the reward will drop. The starting feeder is chosen randomly for each new session.

In [5]:
# Define colors and feeder channel for Arduino output
arduinoBlue = 9         # Blue diodes
arduinoYellow = 10      # Yellow diodes
arduinoRed = 11         # Red diodes
arduinoFeeder1 = 12     # Trigger pulse for feeder1
arduinoFeeder2 = 4      # Trigger pulse for feeder2

# Feeder changes every trial, start feeder randomized
feederID = random.choice([1, 2])

class RealFeeder(Arduino):
    def __init__(self, *args, **kwargs):
        super(RealFeeder, self).__init__(*args, **kwargs)
    def feed(self, feed_id):
        self.digital[feed_id].write(1) 
        time.sleep(.068)
        self.digital[feed_id].write(0) 

        
class FakeFeeder():
    def feed(self, feed_id):
        print("Fake Arduino - feeding %s..." % feed_id)
    def exit(self):
        print("Fake Arduino - exiting...")
        
        
# Initialize REAL Arduino feeder if connected
#board = RealFeeder('COM3')    # May be another COM-Port - in Windows, just check the Hardware Manager

# OR initialize FAKE Arduino feeder for testing purposes. It will just print message here when feeding
board = FakeFeeder()

## Prepare the audio stream

The following cell initiates the audio stream, to which we will later feed our stimuli. The default sample rate is set to 44.1 kHz. The cell also loads sound files with the stimuli. Here, we use short pure tones as stimuli and a silent sound object, which is fed to the audiostream between stimuli. In our setup, we found this to be necessarry to reduce undesired clicking sounds at stimulus on- and offset, even though the sounds are ramped. Whether this will be necessary for you, will strongly depend on your audio hardware. 

The audio stimulation provided by this notebook differs from the MATLAB version in two important aspects: Firstly, the MATLAB version generates the stimuli on the fly, while this notebook uses sound files as input. Feel free to change the code if you prefer the other solution. Secondly, the MATLAB version stimulates at fixed time intervals and the sample rate of the video tracking is locked to the stimulation interval, i.e. high temporal precision in the sound stimulation comes with the cost of lower temporal resolution of the animal tracking. Here, we chose the opposite approach, with the video feed defining the cycle frequency (approx. 14 Hz with the given Camera and a resolution of 800x600 px) and the audio stimulation being locked to the framerate of the camera. Thus, higher temporal resolution of the animal tracking comes with the cost that inter-stimulus intervals cannot freely be chosen, but only be multiple integers (3 or higher) of the mean video frame duration. In the example setup and the code below, we decided for the stimulus to be played every three cycles (approx. every 215 ms). 

The duration of the audio files should not exceed the cycle length.

In [6]:
# Set sample rate for audio output
sd.default.samplerate = 44100
fs = 44100          

# Audio stream
stream = sd.OutputStream(samplerate=fs, channels=1, dtype='float32')

# Cycle counter: sound is played every "delayLength" cycles
commonCycle = 1
delayLength = 3  

# Open sound files
distractorSoundTrial = wavfile.read('./10kHz-short-68.wav')[1]
attractorSoundTarget1 = wavfile.read('./4000Hz-short-68.wav')[1]
silenceSound = wavfile.read('./silence-short-68.wav')[1]

<ipython-input-6-c27cd5fb2fdf>:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  distractorSoundTrial = wavfile.read('./10kHz-short-68.wav')[1]
<ipython-input-6-c27cd5fb2fdf>:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  attractorSoundTarget1 = wavfile.read('./4000Hz-short-68.wav')[1]
<ipython-input-6-c27cd5fb2fdf>:15: WavFileWarning: Chunk (non-data) not understood, skipping it.
  silenceSound = wavfile.read('./silence-short-68.wav')[1]


## Protocol 2

The following cell generates a video object to which the later video feed will be saved. The colours that are defined will later be used for labeling. The labelled video file ("ExperimentID_video.avi") will be saved to the folder containing this notebook for documentation purposes. 

In [7]:
# Define BGR colors
BGR_COLOR = {
    'red': (0,0,255),
     'green': (127,255,0),
     'blue': (255,127,0),
     'yellow': (0,127,255),
     'black': (0,0,0),
     'white': (255,255,255)
}

# Define the codec and create VideoWriter object
videoName = os.path.join(save_to, experiment_id + '_video.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# Make sure that the frame rate of your output appoximately matches 
# the number of cycles per second, to avoid time lapsed output videos
out = cv2.VideoWriter(videoName,fourcc, 15.0, (800,600))


## Capture a background image

The tracking algorithm used in this notebook compares the frames of the video feed during the experiment with an image of the empty arena to later track the position of the largest object in the arena (which usually is your animal). If you are confident in the stability of your video quality, it should suffice to capture the picture once and to skip this cell in the subsequent experiments. However, since this step only takes a few seconds, we recommend to take a new picture of the arena for each new experiment. In the preview of the video feed that will pop-up if you run the next cell, the space outside the arena is masked, so that the camera preview can also be used to check if the camera/arena are still positioned correctly. 

Before taking the picture, make sure that the conditions in your lab (especially the illumination) are the exact same as they will be during the experiments. Once you are happy with the preview of your background image, press "c" to capture the image. It will be saved as "Background.png" to the folder containing this notebook.

This notebook will use the main camera of your system as an input device. If you have more than one camera installed (e.g. on a notebook with internal chat camera), make sure to deactivate all cameras other than the camera of your setup  prior to running the notebook. Also make sure that the video dimensions defined here match you arena dimensions defined above and the video dimensions of the video feeds that will be defined in the subsequent cells.

In [8]:
# Define video capture device (0 = webcam1) to capture background frame
cap = cv2.VideoCapture(0)
# Set picture dimensions
cap.set(3,800)      # Width
cap.set(4,600)      # Height

# Capture Background frame (c = capture)
while(True):
    # Capture frame-by-frame
    ret, img = cap.read()
    img2 = img

    # Display the resulting frame
    imgArena = cv2.circle(img, (cfg['arena_x'], cfg['arena_y']), cfg['arena_radius'], (0,0,255), 2)
    imgArenaStart = cv2.circle(imgArena, (cfg['start_x'], cfg['start_x']), cfg['start_radius'], (255,0,255), 2)

    # Mask the space outside the arena
    mask = np.zeros(shape = img.shape, dtype = "uint8")
    cv2.circle(mask, (cfg['arena_x'], cfg['arena_y']), cfg['arena_radius'], (255,255,255), -1)

    maskedImg2 = cv2.bitwise_and(src1 = img2, src2 = mask)
    imgArenaStart = cv2.bitwise_and(src1 = imgArenaStart, src2 = mask)

    cv2.imshow('Press (c)-to capture the background image',imgArenaStart)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        cv2.imwrite('Background.png',maskedImg2)
        break

# When the background image is captured, release the capture
cap.release()
cv2.destroyAllWindows()

# Loads current background as object img for later use
img = cv2.imread('Background.png',1)

## Prepare the experiment

The following cell will provide another preview of the video feed from the arena. It will allow you to double-check if everything is prepared for the experiment. If so, you can bring your animal and put it into the arena. 

Once you have left the room with your setup and are happy with what you see in the live-feed, hit "c" to close the preview.

In [9]:
# Define video capture device for live-stream (0 = webcam1)
cap2 = cv2.VideoCapture(0)
# Set picture dimensions
cap2.set(3,800)
cap2.set(4,600)

# Show video to see animal leaving the box
while(True):
    # Capture frame-by-frame
    ret, img3 = cap2.read()
    
    cv2.imshow('Press (c)-to continue',img3)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

cap2.release()
cv2.destroyAllWindows()


## Initialize the camera

This cell initializes the camera for the actual tracking and defines some counters and dummy variables needed during the experiment.

In [10]:
# Define video capture device for live-stream (0 = webcam1) and tracking
cap = cv2.VideoCapture(0)
# Set picture dimensions
cap.set(3,800)
cap.set(4,600)

# Mask the space outside the arena
mask = np.zeros(shape = img.shape, dtype = "uint8")
cv2.circle(mask, (cfg['arena_x'], cfg['arena_y']), cfg['arena_radius'], (255,255,255), -1)

# Experiment starts in phase 0 with 0 trials
expPhase = 0   
trialCounter = 0
rewardCounter = 0
frameCounter = 0
trialCountdown = 0
targetCountdown = 0

# Initial values for target area
targetX = 0
targetY = 0

## Protocol 3

The following cell generates an Excel-file to which the essential data (i.e. animal position, positions of the target areas, etc.) from each cycle (video frame) of the experiment will be saved. The Excel-file ("ExperimentID_protocol.xlsx") will be saved to the folder containing this notebook. 

In [11]:
def log_frame_data(args):
    with open(os.path.join(save_to, experiment_id + '_protocol.csv'), 'a') as f:
        f.write(",".join([repr(x) for x in args]) + "\n")

headers = [
    'FrameID',            # Frame ID
    'Time [s]',           # Time stamp
    'Phase',              # Phase of experiment
    'Animal_x',           # X-Coordinate of the subject
    'Animal_y',           # Y-Coordinate of the subject
    'Start_x',            # X-Coordinate of the starting platform
    'Start_y',            # Y-Coordinate of the starting platform
    'Start_rad',          # Radius of the starting platform
    'Target_x',           # X-Coordinate of the target 
    'Target_y',           # Y-Coordinate of the target 
    'Target_rad',         # Radius of the target platform
    'TrialID',            # Trial ID
    'Rewarded Trials [%]',# Percentage of trials rewarded
    'Sound Played'        # sound played
]

## Open a start button

This cell provides a start button. If you run this notebook cell-by-cell, this button is obsolete. However, if you run all cells at once, this is the point of no return. Once you have started the experiment, it cannot be paused until the session criteria are met or it is interrupted manually.

In [12]:
root = tk.Tk()

frame = tk.Frame(root)
frame.pack()

button = tk.Button(frame, text="Start Experiment!", fg="black", command=root.destroy)
button.pack(side=tk.LEFT)

root.mainloop()

## Start the experiment

The final cell contains all the code for animal tracking and hardware control in response to the animals's behavior. We hope that the comments provided in the code suffice to understand the individual steps and to adjust them to your own setup and needs, if necessary.

The experiment will stop automatically, if either one of the following conditions is met:

(1) The pre-defined session duration is reached; <br/>
(2) The pre-definde number of trials is reached; <br/>
(3) The experiment is voluntarily stopped prematurely by hitting "q". 

If you should decide to stop the experiment manually, always use the "q"-button on your keyboard. Just quitting Jupyter/Python will lead to data loss!

In [13]:
# Define and start the experiment timer
expTime = time.time()

# Start the audio stream
stream.start()

# Conditions to be met for the experiment to start and continue
while(cap.isOpened() and trialCounter < cfg['trial_number'] and (time.time()-expTime)<=cfg['session_duration']):
    
    # ---------- render a frame and compute animal position ---------------
    
    # Here you can choose different modes of amplitude modulation by commenting/uncommenting 
    ampMod = (random.randrange(2396,2962,1)/100)**e/10000 # Unbiased Voltage Ratio -5dB
    ### ampMod = random.randrange(5623,10001,1)/10000 # Voltage Ratio -5dB
    ### ampMod = random.randrange(3162,10001,1)/10000 # Power Ratio -5dB
    ### ampMod = 1 # No modulation
    
    ret, frame = cap.read()
    if not ret == True:
        break
        
    maskedFrame = cv2.bitwise_and(src1=frame, src2=mask)

    ## Animal tracking
    # Substracts background from current frame
    subject = cv2.subtract(img, maskedFrame) if cfg['background_color'] == 'T' else cv2.subtract(maskedFrame, img)

    # Converts subject to grey scale
    subjectGray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)

    # Applies blur and thresholding to the subject
    kernelSize = (25,25)
    frameBlur = cv2.GaussianBlur(subjectGray, kernelSize, 0)
    _, thresh = cv2.threshold(frameBlur, 40, 255, cv2.THRESH_BINARY)

    # Finds contours and selects the contour with the largest area
    ###im2, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # If there is no subject, the sreen is blackened, indicating that there is a problem
    # with the tracking or that your animal has escaped.
    # This code block helps when building and testing the setup. During a real experiment,
    # the condition hopefully is never met. 
    if (len(contours) == 0):
        x = 20
        y = 40
        subjectHullCentroid = np.zeros(frame.shape,np.uint8)
        subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)

    # If there is a subject, it is tracked
    else:
        contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
        M = cv2.moments(contour)
        if ((M['m00']) == 0):
            x = 20 if expPhase < 3 else 780
            y = 40 if expPhase < 3 else 580
            subjectHullCentroid = np.zeros(frame.shape,np.uint8)
            subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)
        else:
            x = int(M['m10'] / M['m00'])
            y = int(M['m01'] / M['m00'])
            hull = cv2.convexHull(contour)
            subjectHullCentroid = maskedFrame

        # Draws contour and centroid of the subject
        cv2.drawContours(subjectHullCentroid, [contour], 0, BGR_COLOR['green'], 1, cv2.LINE_AA)
        subjectHullCentroid = cv2.circle(subjectHullCentroid, (x,y), 3, BGR_COLOR['yellow'], -1)

    # Draws the arena contour, the attractor target, the distractor target, and a red / green dot, 
    # signalling that the subject is outside / inside the attractor target area
    dot_colors = {0: 'red', 1: 'green', 2: 'blue', 3: 'green'}
    subjectHullCentroidArena = cv2.circle(subjectHullCentroid, (cfg['arena_x'],cfg['arena_y']), cfg['arena_radius'], (0,0,255), 2)
    subjectHullCentroidArenaStartIn = cv2.circle(subjectHullCentroidArena,(20,20), 10, BGR_COLOR[dot_colors[expPhase]], -6)
    if expPhase > 1:    
        subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(targetX,targetY), cfg['target_radius'], (0, 255, 0), 2)
    else:
        subjectHullCentroidArenaStart = cv2.circle(subjectHullCentroidArena,(cfg['start_x'], cfg['start_x']), cfg['start_radius'], (255,0,255), 2)
    
    # Adds a stopwatch for the experiment duration to the video
    subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartIn,
    '' + str('Time: %.2f' % ((time.time()-expTime))),
    (10,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['white'])

    # Adds a trial duration countdown to the video
    if expPhase > 0:
        subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
        '' + str('Trial: %.2f' % ((cfg['trial_duration']-trialCountdown))),
        (670,590), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['red'] if expPhase == 2 else BGR_COLOR['green'])

    # Adds a target duration countdown to the video
    if expPhase == 3:
        subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
        '' + str('Target: %.2f' % ((cfg['target_duration']-targetCountdown))),
        (670,570), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['green'])

    # Adds the current trial number to the video
    subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
    '' + str('Trial#: %.0f' % (trialCounter)),
    (670,30), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])

    # Adds the current number of collected rewards to the video
    subjectHullCentroidArenaStartInText=cv2.putText(subjectHullCentroidArenaStartInText,
    '' + str('Reward#: %.0f' % (rewardCounter)),
    (670,50), cv2.FONT_HERSHEY_DUPLEX, .5, BGR_COLOR['blue'])

    # Writes the modified frame to the video protocol and shows it in a popup window 
    out.write(subjectHullCentroidArenaStartInText)
    cv2.imshow('Press (q)-to end the experiment',subjectHullCentroidArenaStartInText)

    # Frame ID
    frameCounter += 1

    # Calculates the percentage of successful/rewarded trials
    percentCorrect = 0 if rewardCounter==0 else 100/trialCounter*rewardCounter

    # ---------- experiment logic ---------------
    
    # Phase 0 = Animal just entered the arena or finished a trial
    if expPhase == 0:
        stream.write(silenceSound)  # In phase 0, there is no acoustic stimulation
        soundPlayed = 'false'
        commonCycle = 1
        
        # Checks, if the subject is in the starting/initialization area
        # If so, the protocol proceeds to phase 1 and a timer is started
        if (x-cfg['start_x'])**2 + (y-cfg['start_y'])**2 <= cfg['start_radius']**2:
            expPhase = 1
            startInZone = time.time()
    
    # Phase 1 = Animal is in the starting area
    elif expPhase == 1:
        stream.write(silenceSound)  # In phase 1, there is no acoustic stimulation too
        soundPlayed = 'false'
        commonCycle = 1
        
        # Checks, if the subject is still in the starting/initialization area 
        if (x-cfg['start_x'])**2 + (y-cfg['start_y'])**2 <= cfg['start_radius']**2:
            stopInZone = time.time()

            # Checks, if the time spent in the starting/initialization area exceeds the initiation duration
            # If so, the protocol proceeds to phase 2, the trial timer is started, the designated distractor (trial)
            # sound is played every "delayLength" cycles, and the target areas for the current trial are generated
            if (stopInZone - startInZone) >= cfg['init_duration']:
                expPhase = 2
                startTrial = time.time()

                targetX, targetY = 9000, 9000
                while ( (targetX-cfg['arena_x'])**2 + (targetY-cfg['arena_y'])**2 >= cfg['arena_radius']**2 or               
                           math.sqrt((cfg['start_x']-targetX)**2 + (cfg['start_y']-targetY)**2) <= cfg['start_radius'] + cfg['target_radius'] ):
                    # random angle
                    alpha = 2 * math.pi * random.random()   
                    # random radius
                    r = (cfg['arena_radius'] - 20 - cfg['target_radius']) * math.sqrt(random.random())  
                    # calculating coordinates
                    targetX = int(r * math.cos(alpha) + cfg['arena_x'])  
                    targetY = int(r * math.sin(alpha) + cfg['arena_y'])  

        # If the animal leaves the starting area before the initialization duration is reached, 
        # the protocol goes back to phase 0
        else:   
            expPhase = 0
    
    # Phase 2 = Animal initiated the trial
    elif expPhase == 2:
        stopTrial = time.time()

        # If the maximum trial duration is reached, the trial is terminated and the protocol goes back to phase 0
        if (stopTrial-startTrial) >= cfg['trial_duration']:
            expPhase = 0
            trialCounter += 1
            trialCountdown = 0
            continue

        # Time left for trial successful trial completion
        trialCountdown = (stopTrial - startTrial)

        # Checks, if the animal is in the attractor target area
        # If so, acoustic stimulation switches to the designated attractor stimulus and the protocol 
        # proceeds to phase 3
        if (x-targetX)**2 + (y-targetY)**2 <= cfg['target_radius']**2:
            startInTarget = time.time()
            expPhase = 3

        # If the animal is not in the target areas, the protocol keeps playing back the designated trial
        # distractor stimulus and remains in phase 2
        else:
            if frameCounter % delayLength == 0:
                stream.write((distractorSoundTrial*ampMod))
                soundPlayed = 'true-DistractorTrial'
            else:
                stream.write(silenceSound)
                soundPlayed = 'false'
        
    # Phase 3 = Animal entered the target area
    elif expPhase == 3:

        # Checks, if the animal is still in the attractor target area
        # If so, acoustic stimulation continues with the designated attractor stimulus and the protocol 
        # remains in phase 3          
        if (x-targetX)**2 + (y-targetY)**2 <= cfg['target_radius']**2:
            stopInTarget = time.time()

            if frameCounter % delayLength == 0:
                stream.write((attractorSoundTarget1*ampMod)) 
                soundPlayed = 'true-AttractorTarget1'
            else:
                stream.write(silenceSound)
                soundPlayed = 'false'

            # Checks, if the desired target duration is reached   
            # If so, the subject is rewarded, the trial and reward counters are increased by 1,
            # the target countdown stops, and the protocol goes back to phase 1
            if (stopInTarget - startInTarget) >= cfg['target_duration']:
                trialCounter += 1
                rewardCounter += 1
                targetCountdown = 0

                # Activates the current feeder and switches to the other feeder for the next reward
                board.feed(arduinoFeeder1 if feederID == 1 else arduinoFeeder2)
                feederID = 2 if feederID == 1 else 1
                expPhase = 0
                startTrial = time.time()
                
            # If the desired target duration is not reached, the protocol remains in phase 3 and the
            # countdown continues
            else:
                expPhase = 3
                targetCountdown = (stopInTarget - startInTarget)

        # If the animal has left the attractor target area, the protocol switches to the designated trial
        # distractor stimulus and goes back to phase 2
        else:
            expPhase = 2

    # Writes a new row to the protocol
    log_frame_data([
        frameCounter, (time.time()-expTime), expPhase, 
        x, y, cfg['start_x'], cfg['start_y'], cfg['start_radius'], targetX, targetY, 
        cfg['target_radius'], trialCounter, percentCorrect, soundPlayed
    ])

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# If the session is over or interrupted, all capture and output devices are released, streams are stopped, 
# windows are destroyed, protocol files are saved, and the communication with the Arduino is terminated
cap.release()
out.release()
stream.stop()
cv2.destroyAllWindows()
board.exit()

Fake Arduino - feeding 4...
Fake Arduino - exiting...
